In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
import time
import logging

logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

engine = create_engine('sqlite:///inventory.db')

def ingest_db(file_path, table_name, engine, chunksize=50000):
    # Read CSV in chunks
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        chunk.to_sql(table_name, con=engine, if_exists='append', index=False)

def load_raw_data():
    start = time.time()
    for file in os.listdir('data'):
        if file.endswith('.csv'):
            file_path = os.path.join('data', file)
            logging.info(f'Ingesting {file} into db')
            ingest_db(file_path, file[:-4], engine, chunksize=50000)
    end = time.time()
    total_time = (end - start) / 60
    logging.info('--------------- Ingesting Complete ---------------')
    logging.info(f'Total Time Taken: {total_time:.2f} minutes')

if __name__ == '__main__':
    load_raw_data()
